## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salamiyah,SY,35.0113,37.0532,58.42,34,18,5.12,few clouds
1,1,Chokurdakh,RU,70.6333,147.9167,-10.41,97,83,8.70,broken clouds
2,2,Zhigansk,RU,66.7697,123.3711,-3.10,93,85,14.83,overcast clouds
3,3,Verkhoyansk,RU,67.5447,133.3850,-6.29,99,96,1.21,overcast clouds
4,4,Viedma,AR,-40.8135,-62.9967,61.48,35,30,10.20,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Ocos,GT,14.5094,-92.1933,87.98,65,1,9.89,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,78.17,61,5,11.34,clear sky
8,8,Kavaratti,IN,10.5669,72.6420,81.66,76,50,5.14,scattered clouds
12,12,Vaini,TO,-21.2000,-175.2000,78.96,78,75,10.36,broken clouds
16,16,San Policarpo,PH,12.1791,125.5072,80.80,85,100,17.56,heavy intensity rain
23,23,Boulder City,US,35.9786,-114.8325,85.59,7,0,6.91,clear sky
35,35,Santo Antonio Do Taua,BR,-1.1519,-48.1294,76.95,97,100,3.04,moderate rain
39,39,Puerto Ayora,EC,-0.7393,-90.3518,78.76,96,66,8.01,broken clouds
44,44,Alofi,NU,-19.0595,-169.9187,80.49,78,75,5.75,thunderstorm
52,52,Georgetown,MY,5.4112,100.3354,78.73,92,20,3.44,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# *******not needed: no nulls*******

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Ocos,GT,87.98,clear sky,14.5094,-92.1933,
7,Rikitea,PF,78.17,clear sky,-23.1203,-134.9692,
8,Kavaratti,IN,81.66,scattered clouds,10.5669,72.6420,
12,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
16,San Policarpo,PH,80.80,heavy intensity rain,12.1791,125.5072,
23,Boulder City,US,85.59,clear sky,35.9786,-114.8325,
35,Santo Antonio Do Taua,BR,76.95,moderate rain,-1.1519,-48.1294,
39,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
44,Alofi,NU,80.49,thunderstorm,-19.0595,-169.9187,
52,Georgetown,MY,78.73,few clouds,5.4112,100.3354,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Ocos,GT,87.98,clear sky,14.5094,-92.1933,Auto Hotel El Soñador
7,Rikitea,PF,78.17,clear sky,-23.1203,-134.9692,People ThankYou
8,Kavaratti,IN,81.66,scattered clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
12,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
16,San Policarpo,PH,80.80,heavy intensity rain,12.1791,125.5072,Agapita M. Nicart
...,...,...,...,...,...,...,...
692,Karratha,AU,81.75,clear sky,-20.7377,116.8463,ibis Styles Karratha
693,Mandalgarh,IN,77.95,clear sky,25.2000,75.1000,दुर्गा टेंट हाऊस
696,Kantabanji,IN,79.63,few clouds,20.4833,82.9167,Kamdhenu Borewell And Trading
697,Berbera,SO,80.28,scattered clouds,10.4396,45.0143,IBCO REAL ESTATE PROPERTY


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   217
Country                217
Max Temp               217
Current Description    217
Lat                    217
Lng                    217
Hotel Name             217
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))